In [8]:
import numpy as np
import matplotlib.pyplot as plt

パラメータの最適化は難しい！  
そして確率的勾配降下法はナイーブなので，もう少しスマートな方法を使ってみよう！  

確率的勾配降下法は簡単に実装すると以下のようになる

In [ ]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr
    
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]

SGDを使ってNNのパラメータの更新をする疑似コードを書くと次のようになる

network = TwoLayerNet(...)
optimizer = SGD()

for i in range(10000):
    ...
    x_batch, t_batch = get_mini_batch(...)
    grads = network.gradient(x_batch, t_batch)
    params = network.params
    optimizer.update(params, grads)
    ...

SGDは最適解に行くまでが非効率なことがある．  
ジグザグの動きになって，移動距離が長くなる．  
それを軽減するのが次のMomentum

## Momentum

$$ v \leftarrow \alpha v - \eta \frac{\partial L}{\partial W} $$
$$ W \leftarrow W + v $$

速度の減衰を考慮する更新式．  
vは速度を，αは減速率を示す．  
この最適化手法をクラスであらわすと以下のようになる

In [3]:
class Momentum:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
    
    def update(self, params, grads):
        # 前の速度を使うので，vに速度を記録しておく
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)
        
        for key in params.keys():
            self.v[key] = self.momentum * self.v[key] - self.lr * self.grads[key]
            params[key] += self.v[key]

反動を考慮するので，ジグザグの動きを軽減できる．

## Adagrad

学習係数を減衰させる  
全部の学習係数を減衰させるのではなく，一つ一つのパラメータに対して学習係数を減衰させていくのがAdaGrad  
つまりAdaptive Gradientのこと  

$$ h \leftarrow h + \frac{\partial L}{\partial W} \odot \frac{\partial L}{\partial W} $$
$$ W \leftarrow W - \eta \frac{1}{\sqrt{h}} \frac{\partial L}{\partial W} $$

hは，学習が進むほど大きくなる．$\frac{1}{\sqrt{h}}$が更新時にかかるので，更新される重みは学習が進むほど小さくなっていく  

無限に学習を行うと更新量が0になってしまう．  
これを改善した手法にRMSPropがある．こちらは，新しい勾配の情報が大きく反映されるように加算していく．  
指数関数的に過去の勾配のスケールを減少させる「指数移動平均」を用いる．

In [4]:
class AdaGrad:
    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None
    
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
        
        for key in params.keys():
            self.h += grads[key] * grads[key]
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7) # ゼロ除算を防ぐ

めっちゃ効率的に最小値に向かう！

## Adam

AdaGrad + Momentum  
3つのハイパーパラメータを使用した複雑めな手法．多くの場合うまくいく設定値がある．  

以上4つの手法を紹介したが，どれが一番適しているかは，解くべき問題によって異なるので，いろいろ試してみるのがよい．  
最近はAdamを使う研究者が多いらしい

MNISTデータセットに対して，以上の更新手法を比較してみると，SGDよりも他の手法が早く，  
特にAdaGradはやや他の手法より早い．最終的な認識性能も高くなる．

# 重みの初期値

重みの初期値がすべて0なのはよくない．  
順伝搬で次の層に0が渡されれば，逆伝搬時に返ってくる値が全て0になってしまう．  
すると，せっかくたくさんのパラメータを持つ意味がなくなってしまう．  
そのため，重みの対称的な構造を崩す，すなわち重複する値を持たないようにすることが必要．  
従って，ランダムな初期値が必要になる．

活性化関数を通った後の出力データの分布，アクティベーション分布を観察してみよう．  
5層NN，活性化関数にシグモイド関数を使ったものに，ランダムな入力データを流し，ヒストグラムで描画してみる．

In [9]:
# 実験プログラムの一部

def sigmoid(x):
    return 1/(1 + np.exp(-x))

x = np.random.randn(1000, 100) # 1000個のデータ
node_num = 100
hidden_layer_size = 5
activations = {}

for i in range(hidden_layer_size):
    if i != 0:
        x = activations[i-1]
    
    w = np.random.randn(node_num, node_num) * 1 # 掛けた値が標準偏差になる
    
    z = np.dot(x, w)
    a = sigmoid(z)
    activations[i] = a

上記のコードは標準偏差1のガウス分布を使って重みを生成しているが，この標準偏差を変えたらどうなるだろうか？  

標準偏差1のとき，どの層も出力は0か1付近に固まることが分かる．  
0と1に固まった出力は，次の層の出力も同じように0と1に固めてしまう．  
この問題を**勾配消失**と呼び，層が深くなるほど深刻になる．  

標準偏差0.01のとき，アクティベーションは0.5に固まる．これもよくない．  
これはすなわち，「表現力の制限」という問題になっている．  

各層のアクティベーションの分布は，適度な広がりを持つことが求められる．  
そのほうが，学習が効率的に行える．

次に，Xavierの初期値を使ってみる．  
これは，$\frac{1}{\sqrt{n}}$の標準偏差を持つものである．nはノードの数である．  
この初期値を使うと，前よりも適度な広がりを持つアクティベーション分布になる．

上位の層になるほどいびつな形になるが，活性化関数に，似た形のtanhを使うと改善される．  
なお，活性化関数に用いる関数は，原点対象であることが望ましい．

Xavierの初期値は，左右対称で中央付近が線形関数としてみなせるSigmoid関数やtanh関数に適している．  
一方，初期値を$\sqrt{\frac{2}{n}}$とする「Heの初期値」は，ReLUに特化している．

MNISTデータセットはで試してみると，  
標準偏差0.01 ＝＞ 学習全然進まない  
Xavierの初期値＋Sigmoid ＝＞ 学習よく進む  
Heの初期値＋ReLU ＝＞ Xavierよりも学習良く進む

結論：初期値は大事！

# Batch Normalization  
各層のアクティベーションの分布を強制的に調整しようとする  

- 学習を早く進行させられる
- 初期値にあまり依存しない
- 過学習を抑制する

データ分布の正規化を行う，「Batch Normレイヤ」をAffineレイヤとReLUレイヤの間に挿入する．
Batch Normレイヤはデータの分布を平均0，分散1になるよう調整する．
$$\hat{x_i} \leftarrow \frac{x_i - \mu_B}{\sqrt{\sigma^2_B + \epsilon}}$$

さらに，正規化されたデータに対して固有のスケール(傾き)とシフト(切片)で変換を行う．  
$$y_i \leftarrow \gamma \hat{x_i} + \beta$$

$\gamma = 1, \beta = 0$からスタートして，学習の進度に適した値に調整される． 

逆伝搬はちょいめんどい．  

実際動作させてみると，BatchNormを使った方が早く，精度もいい．  
これにより，初期値にロバスト(あまり依存しない)学習を実現できる．  

## 過学習  
訓練データだけに適応しすぎ，他のデータにうまく対応できなくなること

### 過学習の原因  
- モデルのパラメータが多く，表現力が高い．
- 訓練データが少ない．  

学習データを60000から300にしてみる．  
すると，100エポックを過ぎたあたりから，訓練データにはほぼ100%適合し，テストデータには60%ほどで頭打ちになるという結果になる．  

### 過学習の抑制

#### weight decay  
荷重減衰  
大きな重みをもつことにペナルティを課す  
過学習は重みが大きくなることによって発生しやすい  
例えば，L2ノルム(重さの2乗和)を損失関数に加えることで正則化を行う．
$$ Loss(t) + \frac{1}{2} \lambda W^2 $$
$\lambda$はペナルティの強さ，$ \frac{1}{2} $は微分の結果を調整するための値, tは学習のラウンド  
これより，逆伝搬の時には$\lambda W$が加算される．  
  
結果，訓練データとテストデータの認識精度の隔たりを軽減できる．

#### Dropout  
ニューロンをランダムに消して学習を行う  

これもまた，訓練データとテストデータの認識精度の隔たりを軽減できる．  

これは，アンサンブル学習を疑似的に一つのネットワークで実現しているといえる．

## ハイパーパラメータの検証  
パラメータ： 重みやバイアス  
ハイパーパラメータ： ニューロン数，バッチサイズ，学習係数，Weight decayなど  

テストデータを使ってハイパーパラメータの値を調整してはいけない．  
テストデータに対して過学習することもあり得るから．  

よって，テストデータとは別にハイパーパラメータ調整用の「検証データ」と呼ばれるものが必要になる．  
訓練データの中から20%を分離したりする．データが並んでたりするので分離前にシャッフルすること．  

1. ハイパーパラメータの範囲をざっくりと設定する．
    - 0.001 ~ 10^3など
1. その範囲から，ランダムに値をサンプリングする．
1. その値を使って学習を行い，検証データで認識精度を評価する．
1. ステップ2, 3を100回ほど繰り返し，それらの認識精度の結果がよかったものを参考に，ハイパーパラメータの範囲を狭めていく．

ベイズの定理を使って効率よく最適化を行う`ベイズ最適化`なども使える．

## まとめ  
- パラメータの更新方法
    - SGD: 確率的勾配降下法，ランダムにミニバッチを生成し，学習
        - ナイーブ，解にジグザグに向かう
    - Momentum: 速度の減衰を考慮に入れた更新式を使う
        - SGDよりは早く解に向かう
    - AdaGrad: Adaptive Gradient, 一つ一つのパラメータに対して学習が進むたびに学習係数を減衰させていく
        - なかなか効率的
    - Adam: AdaGrad + Momentum
        - 3つのハイパーパラメータを持ち，なかなか複雑だが，これが最近は多く使われている．
  
- 重みの初期値の与え方
    - 勾配消失問題: どの層も出力が0, 1に固まると，次の層の出力も0, 1になってしまい，いい結果が得られない問題．
    - Xavierの初期値: $\frac{1}{\sqrt{n}}$の標準偏差を持つ
        - tanhやSigmoid，恒等関数のような原点対象の関数に強い
    - Heの初期値: 初期値を$\sqrt{\frac{2}{n}}$とする
        - ReLUに特化
      
- Batch Normalization
    - 活性化関数の前後で，出力の平均を0，分散を1にすることで，学習を早め，精度を高め，過学習を防ぐ．
  
- 過学習抑制
    - Weight decay: L2ノルムなどの正則化項を加える
    - Dropout: ランダムにニューロンを消去し，疑似アンサンブル学習
  
- ハイパーパラメータの探索
    - 検証データを使って認識精度が良かったときのパラメータの範囲を観察し，狭めていく．
    - ベイズ最適化